이전에 split했었던 문서에 embed 작업을 해보자.

그 전에 먼저 embedding model에 대해 살펴보자.

OpenAI가 가지고 있고, LangChain이 지원하는 거다.

In [ ]:
from langchain.embeddings import OpenAIEmbeddings

embedder = OpenAIEmbeddings()

embedder.embed_query("Hi")

```python
embedder.embed_query("Hi")
```
위 코드를 실행하면 embed 작업을 수행한다.

그럼 'Hi'를 표현하는 벡터를 얻게 될 것이다.

차원이 몇 개 인지 확인해 보자.

In [2]:
vector = embedder.embed_query("Hi")
len(vector)

1536

한 개의 단어 Hi를 위한 vector의 차원은 1536개 이다. (차원)

이번에는 문서들을 embedding 해보자.  
이것들은 string의 배열로 되어있어야 한다.


In [4]:
# 이것들은 string의 배열로 되어있어야 한다.
vector = embedder.embed_documents([
    'hi',
    'how',
    'are',
    'you longer sentence because',
])
# print(vector)
print(len(vector), len(vector[0]))

4 1536


vector에 입력해 준 4개의 각 문서에 하나씩 할당된 것이다.

또 이들 각각은 같은 차원을 갖고 있다.

즉, 위에서 우리는 4개의 벡터가 있고, 각 벡터는 1536개의 차원을 갖고 있다.

---

이제 실제로 우리 문서를 embed 해보자.

위에서처럼 직접적으로 하진 않을 것이다. 

왜냐하면 문서 embed를 반복해서 수행하고 싶지 않기 때문이다.

<b>대신, 우리는 그 embed들을 저장해 줄거다.</b>

이 방법으로 코스트를 줄일 수 있다.

그리고 Langchain은 Embedding한 것들을 캐싱하는 기능을 제공하고 있다.

다시 말하지만, embed를 매번 코드를 실행할 때마다 하는 것은 좋은게 아니다.

한 번 document embed를 해놓으면, 문서가 바뀌지 않는 이상 embeddings도 바뀌지 않는다.

---  



이제 vector store에 대해 알아보자.

vector store는 일종의 "데이터베이스"라고 생각할 수 있다.

벡터 공간에서 검색을 할 수 있게 해준다.

그러니까, 우리가 벡터들을 만들고나서 그것들을 캐시해주고 vector store에 그 벡터들을 넣어주면,

우리가 검색을 할 수 있다.

관련있는 문서들만 찾아낼 수 있게 되는 것이다.

LangChain 은 다양한 Vector Store을 지원하고 있다.

우리는 그 중 우리 컴퓨터에서 직접 실행되는 Chroma 를 사용해 본다.

---

Chroma를 생성해 보자.

Chroma에는 분할된(splitted) 문서와 openAI embeddings model을 전달한다.

In [3]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings

# Chroma
from langchain.vectorstores import Chroma

splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100,
)

loader = UnstructuredFileLoader("./files/chapter_one.pdf")

# 분할된 문서
docs = loader.load_and_split(text_splitter=splitter)

# 임베딩 모델
embeddings = OpenAIEmbeddings()

# Chroma 초기화 : .from_documents()
vectorstore = Chroma.from_documents(docs, embeddings)

Chroma 는 이렇게 from_documents를 하면 초기화 할 수 있다.

vectorstore에서 유사도 검색을 수행해보자(vectorstore.similarity_search)

이제 벡터 공간에 대한 검색을 시작할 수 있게 되었다.

In [4]:
vectorstore.similarity_search("where does winston live")

[Document(page_content="The Ministry of Love was the really frightening one. There were no windows in it at all. Winston had never\nbeen inside the Ministry of Love, nor within half a kilometre of it. It was a place impossible to enter except on\nofficial business, and then only by penetrating through a maze of barbed-wire entanglements, steel doors,\nand hidden machine-gun nests. Even the streets leading up to its outer barriers were roamed by gorilla-faced\nguards in black uniforms, armed with jointed truncheons.\nWinston turned round abruptly. He had set his features into the expression of quiet optimism which it was\nadvisable to wear when facing the telescreen. He crossed the room into the tiny kitchen. By leaving the\nMinistry at this time of day he had sacrificed his lunch in the canteen, and he was aware that there was no\nfood in the kitchen except a hunk of dark-coloured bread which had got to be saved for tomorrow's breakfast.\nHe took down from the shelf a bottle of colourl

보이는 것처럼 실행의 결과로 decument들이 반환되었다.

값을 담을 results 변수를 만들어보자.

len 함수에 results를 넣고 실행해서 results에 든 documents의 수를 알아보자.

In [5]:
results = vectorstore.similarity_search("where does winston live")
len(results)

4

보다시피 4개의 document를 받아왔다.

그러니까 우리는 모든 문서들을 벡터로 바꾸었고, 

그 벡터들을 넣은 vectorstore를 활용해서 검색을 할 수 있게 되었다.

이제 이 query("where does winston live")와 비슷한 문서를 찾을 수 있게 된 것이다.

이제 결과를 살펴보자.


In [6]:
results

[Document(page_content="The Ministry of Love was the really frightening one. There were no windows in it at all. Winston had never\nbeen inside the Ministry of Love, nor within half a kilometre of it. It was a place impossible to enter except on\nofficial business, and then only by penetrating through a maze of barbed-wire entanglements, steel doors,\nand hidden machine-gun nests. Even the streets leading up to its outer barriers were roamed by gorilla-faced\nguards in black uniforms, armed with jointed truncheons.\nWinston turned round abruptly. He had set his features into the expression of quiet optimism which it was\nadvisable to wear when facing the telescreen. He crossed the room into the tiny kitchen. By leaving the\nMinistry at this time of day he had sacrificed his lunch in the canteen, and he was aware that there was no\nfood in the kitchen except a hunk of dark-coloured bread which had got to be saved for tomorrow's breakfast.\nHe took down from the shelf a bottle of colourl

보이는것처럼 우리가 vectorstore, vector database에서 질문(query)을 검색하면

그 질문과 관련있는 문서들을 반환받게 될 것이다.

만약 받은 문서가 엄청 큰 문서들이라면, 엄청 큰 문서들이 LLM에 전달될거고,

그만큼 비용을 지불하게 될 것이다.

그러니까 작은 부분으로 분할(split)해주는 건 아주 좋은 아이디어이다.

---

이제 이 embeddings을 캐싱해주자.

왜냐하면 다시 실행하면 이전에 했던 것들은 사라지고, 재실행하면 다시 계산해야되기 때문이다.

먼저 CacheBackedEmbedding을 import 해온다.

LocalFileStore을 사용해서 캐시 경로를 설정하자. 이곳에 embedding 파일을 저장한다.

캐시 폴더를 만들고 cached embeddings을 생성해보자.

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.storage import LocalFileStore

# 캐시 경로
cache_dir = LocalFileStore('./.cache/')



splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100,
)

loader = UnstructuredFileLoader("./files/chapter_one.pdf")

docs = loader.load_and_split(text_splitter=splitter)

embeddings = OpenAIEmbeddings()

# .from_bytes_store 메서드는 임베딩 작업을 위해 필요한 embedder의 입력을 요구한다.
cached_embeddings = CacheBackedEmbeddings.from_bytes_store(
    embeddings, cache_dir,
)

vectorstore = Chroma.from_documents(docs, embeddings)


